In [9]:
from docx import Document
import pandas as pd
import re
import math

In [89]:
def lookup_fn2(df, key_col, key_val):
    try:
        result_df = df.loc[df[key_col] == key_val].iloc[0]
        return result_df
    except IndexError:
        return pd.DataFrame()

In [93]:
def GetValidAIPForCollection(origDf, collectionCSVNamesToName):
    print("new")
    bagRegex = r"^Bag-(.*)_(.*).zip$"
    bagRegex2 = r"^(.*)_(.*)_foxml_atomzip.zip$"
    df = pd.read_csv("../bags_validation_report.csv")
    numBagPID = df.shape[0]
    numObjects = origDf.shape[0]
    numValid = 0
    numInvalid = 0
    totalAIP = 0
    totalValidAIP = 0
    pidCol = origDf["PID"]
    collectionCol = origDf["isMemberOfCollection"]
    collectionToNumValid = {}
    for index, row in df.iterrows():
        bag = row["bag_name"]
        isBagValid = False
        if re.match(bagRegex, bag):
            pid = re.search(bagRegex, bag).group(1)+":"+re.search(bagRegex, bag).group(2)
            isBagValid = True
        elif re.match(bagRegex2, bag):
            pid = re.search(bagRegex2, bag).group(1)+":"+re.search(bagRegex2, bag).group(2)
            isBagValid = True
        if isBagValid:
            origRow = lookup_fn2(origDf, "PID", pid)
            if (not origRow.empty):
                collectionRaw = origRow["isMemberOfCollection"]
                if pd.notna(collectionRaw) and pd.isna(origRow["isPageOf"]) and pd.isna(origRow["isConstituentOf"]):
                    collection = collectionCSVNamesToName[collectionRaw]
                    if collection not in collectionToNumValid:
                        collectionToNumValid[collection] = [0,0]
                    collectionToNumValid[collection][0]+=1
                    totalAIP+=1
                    valid = row["validation_status"]
                    if valid:
                        collectionToNumValid[collection][1]+=1
                        totalValidAIP+=1
            else:
                print("not found",pid)
    return collectionToNumValid, totalAIP, totalValidAIP